In [39]:
# %matplotlib inline

from __future__ import annotations
from cycler import cycler
import json
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerTuple
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
import shutil
from sklearn.model_selection import ParameterGrid
import tensorflow as tf
import tensorflow.keras as keras
from typing import Callable, Literal

sns.set() # Use seaborn themes.

In [40]:
# Here define some root paths for saving content.
roots = dict(
    dataset = Path('./dataset/'),
    images = Path('./images/'),
    tuning = Path('./tuning/'),
    tables = Path('./tables/'),
)
for k,r in roots.items(): # Create directories if they do not already exist.
    if not r.exists():
        r.mkdir(exist_ok=True)

In [41]:
def load_history(history_path: str):
    """Load model history from file."""
    return pd.read_csv(history_path)

def hypertune_load_parametergrid(tuning_root: Path) -> list[dict]:
    parameter_grid_path = tuning_root/'parameter_grid.json'
    with open(parameter_grid_path, 'r') as f:
        grid = json.load(f)
    return grid


def hypertune_load_history(tuning_root: Path) -> list[pd.DataFrame]:
    # Load parameter grid so that we know how many models there are.
    grid = hypertune_load_parametergrid(tuning_root)

    # Iterate over parameter grid to load history for each model in the correct order.
    histories: list[list[pd.DataFrame]] = [] # History lists for each model.
    for i, p in enumerate(grid):
        cur_model_name = f"model_{i}"
        history_path = tuning_root/f"{cur_model_name}_history.csv"
        hist = load_history(history_path)
        histories.append(hist)
    return histories

In [42]:
tuning_root = roots['tuning']/"gan-hypertune"
grid = hypertune_load_parametergrid(tuning_root)
histories = hypertune_load_history(tuning_root)

In [43]:
histories[0].head()

,epoch,d_accuracy,d_loss,g_accuracy,g_loss,g_mse,loss,val_d_accuracy,val_d_loss,val_g_accuracy,val_g_loss,val_g_mse,val_loss
0,0,0.750086,20.212833,0.162676,2.787466,0.163027,11.500150,0.000000,3.848210,0.048839,2.732713,0.236527,3.290461
1,1,0.782929,7.554433,0.126665,2.155943,0.274054,4.855188,0.287285,1.314022,0.104591,2.131450,0.180038,1.722736
2,2,0.879350,9.079504,0.048779,3.450586,0.277940,6.265045,0.230230,2.654163,0.123409,3.634990,0.137055,3.144577
3,3,0.935870,5.313097,0.031573,3.272716,0.277990,4.292906,0.241706,2.702956,0.008946,3.401550,0.219423,3.052253
4,4,0.924646,4.393957,0.037460,3.655999,0.292703,4.024978,0.234116,3.820737,0.104760,3.881640,0.193627,3.851189


In [56]:
def make_latex(
    df: pd.DataFrame,
    label: str = None,
    latex_config: dict = {},
    ):
    latex_df = df.copy()
    styler = latex_df.style
    styler = styler.hide(axis=0) # Hide the index.
    styler = styler.format(str, escape='latex') # Default is to convert all cells to their string representation.

    latex_df.columns = latex_df.columns.map(lambda x: x.replace('_', '\_')) # Escape the header names too.

    latex_string = styler.to_latex(
        # buf=latex_path,
        hrules=True,
        label=label,
        **latex_config,
    )

    return latex_string

In [75]:
df = pd.DataFrame(grid)
df.index.rename('Model', inplace=True)
df.reset_index(inplace=True)
df.drop(['D_layer_type', 'G_layer_type', 'D_model_type', 'G_model_type', 'latent_dim', 'optim', 'feat_len', 'k', 'beta_1', 'beta_2', 'window_len'], axis=1, inplace=True)

latex_path = roots['tables']/'gan_hypertune_params_unique.tex'
latex_string = make_latex(df)
with open(latex_path, 'w') as f:
    f.write(latex_string)

df.head()

,Model,D_hidden_units,D_n_layers,G_hidden_units,G_n_layers,lr
0,0,24,3,24,3,0.0010
1,1,24,3,24,3,0.0001
2,2,24,3,24,6,0.0010
3,3,24,3,24,6,0.0001
4,4,24,3,72,3,0.0010


In [76]:
df = pd.DataFrame(grid)
df = df[['D_model_type', 'D_layer_type', 'feat_len', 'latent_dim', 'window_len', 'k', 'beta_1', 'beta_2', 'optim']]
df.rename(columns={'D_layer_type': 'layer_type', 'D_model_type': 'model_type'}, inplace=True)
df = df.iloc[[0]]
df = df.T
df.reset_index(inplace=True)
df.columns = ['Parameter', 'Value']

latex_path = roots['tables']/'gan_hypertune_params_common.tex'
latex_string = make_latex(df)
with open(latex_path, 'w') as f:
    f.write(latex_string)

df.head()

,Parameter,Value
0,model_type,rnn
1,layer_type,gru
2,feat_len,2
3,latent_dim,128
4,window_len,24
